In [13]:
import cv2
import os

INPUT_DIR = "fotos"
OUTPUT_DIR = "fotos_binarizadas"

os.makedirs(OUTPUT_DIR, exist_ok=True)

for subdir, _, files in os.walk(INPUT_DIR):
    rel_path = os.path.relpath(subdir, INPUT_DIR)
    out_subdir = os.path.join(OUTPUT_DIR, rel_path)
    os.makedirs(out_subdir, exist_ok=True)

    for file in files:
        if file.lower().endswith(".jpg"):
            in_path = os.path.join(subdir, file)
            out_path = os.path.join(out_subdir, file)

            img = cv2.imread(in_path)

            if img is None:
                print(f"⚠️ No se pudo leer {in_path}")
                continue

            # 3) Convertir a escala de grises
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # 4) Binarizar con Otsu
            _, binary = cv2.threshold(
                gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
            )

            # 5) Guardar resultado
            cv2.imwrite(out_path, binary)
            print(f"✅ Procesada: {in_path} -> {out_path}")

print("✨ Proceso completado")

✅ Procesada: fotos/76bfaf67-5fdb-44b3-913b-c0d2d7003721/28.jpg -> fotos_binarizadas/76bfaf67-5fdb-44b3-913b-c0d2d7003721/28.jpg
✅ Procesada: fotos/76bfaf67-5fdb-44b3-913b-c0d2d7003721/29.jpg -> fotos_binarizadas/76bfaf67-5fdb-44b3-913b-c0d2d7003721/29.jpg
✅ Procesada: fotos/76bfaf67-5fdb-44b3-913b-c0d2d7003721/22.jpg -> fotos_binarizadas/76bfaf67-5fdb-44b3-913b-c0d2d7003721/22.jpg
✅ Procesada: fotos/76bfaf67-5fdb-44b3-913b-c0d2d7003721/23.jpg -> fotos_binarizadas/76bfaf67-5fdb-44b3-913b-c0d2d7003721/23.jpg
✅ Procesada: fotos/76bfaf67-5fdb-44b3-913b-c0d2d7003721/27.jpg -> fotos_binarizadas/76bfaf67-5fdb-44b3-913b-c0d2d7003721/27.jpg
✅ Procesada: fotos/76bfaf67-5fdb-44b3-913b-c0d2d7003721/26.jpg -> fotos_binarizadas/76bfaf67-5fdb-44b3-913b-c0d2d7003721/26.jpg
✅ Procesada: fotos/76bfaf67-5fdb-44b3-913b-c0d2d7003721/24.jpg -> fotos_binarizadas/76bfaf67-5fdb-44b3-913b-c0d2d7003721/24.jpg
✅ Procesada: fotos/76bfaf67-5fdb-44b3-913b-c0d2d7003721/30.jpg -> fotos_binarizadas/76bfaf67-5fdb-44b3-9

In [11]:
import cv2
import os

INPUT_DIR = "fotos"
OUTPUT_DIR = "fotos_binarizadas"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Tamaño del bloque que quieres reemplazar
BLOCK_W, BLOCK_H = 200, 30

for subdir, _, files in os.walk(INPUT_DIR):
    rel_path = os.path.relpath(subdir, INPUT_DIR)
    out_subdir = os.path.join(OUTPUT_DIR, rel_path)
    os.makedirs(out_subdir, exist_ok=True)

    for file in files:
        if file.lower().endswith(".jpg"):
            in_path = os.path.join(subdir, file)
            out_path = os.path.join(out_subdir, file)

            # Leer imagen en escala de grises
            img = cv2.imread(in_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue

            h, w = img.shape

            # --- Recortar bloque de la esquina superior derecha ---
            x1_src = w - BLOCK_W
            y1_src = 0
            x2_src = w
            y2_src = BLOCK_H
            patch = img[y1_src:y2_src, x1_src:x2_src].copy()

            # --- Pegar en la esquina superior izquierda ---
            img[0:BLOCK_H, 0:BLOCK_W] = patch

            # --- Binarizar con Otsu ---
            _, binary = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # Guardar resultado
            cv2.imwrite(out_path, binary)
            print(f"✅ {in_path} -> {out_path}")

print("✨ Proceso completado")

✅ fotos/76bfaf67-5fdb-44b3-913b-c0d2d7003721/28.jpg -> fotos_binarizadas/76bfaf67-5fdb-44b3-913b-c0d2d7003721/28.jpg
✅ fotos/76bfaf67-5fdb-44b3-913b-c0d2d7003721/29.jpg -> fotos_binarizadas/76bfaf67-5fdb-44b3-913b-c0d2d7003721/29.jpg
✅ fotos/76bfaf67-5fdb-44b3-913b-c0d2d7003721/22.jpg -> fotos_binarizadas/76bfaf67-5fdb-44b3-913b-c0d2d7003721/22.jpg
✅ fotos/76bfaf67-5fdb-44b3-913b-c0d2d7003721/23.jpg -> fotos_binarizadas/76bfaf67-5fdb-44b3-913b-c0d2d7003721/23.jpg
✅ fotos/76bfaf67-5fdb-44b3-913b-c0d2d7003721/27.jpg -> fotos_binarizadas/76bfaf67-5fdb-44b3-913b-c0d2d7003721/27.jpg
✅ fotos/76bfaf67-5fdb-44b3-913b-c0d2d7003721/26.jpg -> fotos_binarizadas/76bfaf67-5fdb-44b3-913b-c0d2d7003721/26.jpg
✅ fotos/76bfaf67-5fdb-44b3-913b-c0d2d7003721/24.jpg -> fotos_binarizadas/76bfaf67-5fdb-44b3-913b-c0d2d7003721/24.jpg
✅ fotos/76bfaf67-5fdb-44b3-913b-c0d2d7003721/30.jpg -> fotos_binarizadas/76bfaf67-5fdb-44b3-913b-c0d2d7003721/30.jpg
✅ fotos/76bfaf67-5fdb-44b3-913b-c0d2d7003721/25.jpg -> fotos_bin